In [ ]:
import json
import test_environment
import csv
import os

tasks = ["Match","Pattern","Bills","Triangle","Graphs","SquareGrid","Database","ContactTracer"]

versions = ["gpt-4o-2024-05-13", "gpt-4o-mini-2024-07-18"]

seeds = range(50)

client = test_environment.openai.OpenAI()

for version in versions:
    for task_name in tasks:
        for seed in seeds:
            print(task_name, seed)

            with open("task_collections.json", "r", encoding="utf-8") as json_file:
                data = json.load(json_file)

            task_description_collection = data.get("task_description_collection", {})
            template_collection = data.get("template_collection", {})

            message, system_fingerprint, system, prompt = test_environment.generate_message(
                client=client, 
                version=version, 
                seed=seed, 
                task_description=task_description_collection[task_name], 
                template=template_collection[task_name], 
            )

            print(message)
        
            code = test_environment.extract_java_code(message)

            if code == None:
                code = message.replace('"""','')

            score, result = test_environment.test_java_code(class_name=task_name, code=code, verbose=True)

            print(score, result)

            code_single_line = code.replace('\n', ' ')
            system_single_line = system.replace('\n', ' ')
            prompt_single_line = prompt.replace('\n', ' ')
            result_single_line = str(result).replace('\n', ' ')
            char_count = len([char for char in code if not char.isspace()])

            tasks_names = ["Match", "Pattern", "Triangle", "Bills", "Graphs", "SquareGrid", "Database", "ContactTracer"]
            task_to_week = {tasks_names: week for tasks_names, week in zip(tasks, range(4, 4 + len(tasks_names)))}

            week = task_to_week[task_name]

            file_path = "test.csv"

            file_exists = os.path.isfile(file_path)

            with open(file_path, "a", newline='', encoding="utf-8") as file:
                writer = csv.writer(file, quotechar='"', quoting=csv.QUOTE_MINIMAL)

                if not file_exists or os.stat(file_path).st_size == 0:
                    writer.writerow(["Week", "Task Name", "Version", "Seed", "System Fingerprint", "Score", "Result", "Char Count", "Code", "System", "Prompt"])

                writer.writerow([week, task_name, version, seed, system_fingerprint, score, result_single_line, char_count, code_single_line, system_single_line, prompt_single_line])